In [2]:
from numpy import vstack
from pandas import read_csv

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import torch

# Criando tensor torch a partir de um list

In [3]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
print(type(t_c))
t_c = Tensor(t_c)
print(type(t_c))

<class 'list'>
<class 'torch.Tensor'>


# Criando tensor torch de a partir de numpy

In [4]:
data = [[1, 2],[3, 4]]
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

NameError: name 'np' is not defined

# criando tensor de outro tensor torch

In [5]:
x_ones = torch.ones_like(x_np) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_np, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

NameError: name 'x_np' is not defined

# Atributo de tensor
* shape
* dtype
* device: gpu ou cpu

In [32]:
tensor = torch.rand(3,4)

print(f"Shape : {tensor.shape}")
print(f"Datatype : {tensor.dtype}")
print(f"Device do tensor : {tensor.device}")

Shape : torch.Size([3, 4])
Datatype : torch.float32
Device do tensor : cpu


# Verificando se GPU está disponível

In [33]:
torch.cuda.is_available()

True

# Armazenando device no tensor


In [34]:
tensor2 = tensor.to('cuda')
print(f"Shape : {tensor2.shape}")
print(f"Datatype : {tensor2.dtype}")
print(f"Device do tensor : {tensor2.device}")

RuntimeError: CUDA error: out of memory

# In-place: Operações que têm um sufixo _

In [46]:
x1 = [[1, 2],[3, 4]]
x = Tensor(x1)
print(x)
x.t_()
print(x)

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 3.],
        [2., 4.]])


# Convertendo tensor para numpy

In [48]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")
print(type(n))

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]
<class 'numpy.ndarray'>


# Named Tensors

In [53]:
imgs = torch.randn(1, 2, 2, 3, names=('N', 'C', 'H', 'W'))
print(imgs.names)
print(imgs)
print(imgs.shape)

('N', 'C', 'H', 'W')
tensor([[[[ 0.2238,  0.3222,  0.8849],
          [-0.3822,  1.4070, -0.3666]],

         [[-0.1926,  2.2001, -1.1293],
          [ 1.1009, -1.0890,  0.0486]]]], names=('N', 'C', 'H', 'W'))
torch.Size([1, 2, 2, 3])


## Dataset
* Dataset para prever se existe uma estrutura na atmosfera ou se não de acordo com retornos de um radar (Ionosphere Dataset Description)
* Y categorias b - bad ou g - good

In [6]:
path = 'ionosphere.csv'
path = 'pima-indians-diabetes.csv'
df = read_csv(path)
df

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [7]:
df.values[:, :-1]

array([[1.00e+00, 8.50e+01, 6.60e+01, ..., 2.66e+01, 3.51e-01, 3.10e+01],
       [8.00e+00, 1.83e+02, 6.40e+01, ..., 2.33e+01, 6.72e-01, 3.20e+01],
       [1.00e+00, 8.90e+01, 6.60e+01, ..., 2.81e+01, 1.67e-01, 2.10e+01],
       ...,
       [5.00e+00, 1.21e+02, 7.20e+01, ..., 2.62e+01, 2.45e-01, 3.00e+01],
       [1.00e+00, 1.26e+02, 6.00e+01, ..., 3.01e+01, 3.49e-01, 4.70e+01],
       [1.00e+00, 9.30e+01, 7.00e+01, ..., 3.04e+01, 3.15e-01, 2.30e+01]])

In [8]:
df.values[:, -1]

array([0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 0.

## Criando Rede Neural com Pytorch
* Preparar os dados
* definir o modelo
* treinar o modelo
* avaliar o modelo
* Predição


## Função que utiliza a classe dataset para realizar a preparação de dados

* classe especializada para controlar acesso aos dados
* Transformar entrada e saída para modo numérico
* Configurações no dataset
    * normalização
    * codificação
    *etc
    
* Normalmente é realizado override das funções:
    * __len__() retorna tamannho do dataset
    * __getitem__() obtém uma amostra pelo índice


 Classe para controlar acesso aos dados

In [9]:
class CSVDataset(Dataset):
    def __init__(self, path):
        df = read_csv(path)

        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        
        self.X = self.X.astype('float32')

        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # quantas linhas tem no dataset?
    def __len__(self):
        return len(self.X)
 
    # obtem uma linha do dataset
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # retorna base para treino e teste
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

## Cria estruturas DataLoader para padronizar a leitura de dados
* DataLoader é uma rotina que cria um leitor de dados de entrada em batch. Dessa forma, não é necessário deixar a base inteira carregada na ram antes de comecar o processamento

In [10]:
def prepare_data(path):
    # Carrega Dataset
    dataset = CSVDataset(path)
    # realiza split
    train, test = dataset.get_splits()
    # monta data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    #test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    test_dl = DataLoader(test, batch_size=32, shuffle=False)
    
    return train_dl, test_dl

## Função para treino do modelo

* Extendendo classo torch.nn.Module
    * Modelos genéricos de rede Neural

* contrutor da classe define as camadas do modelo 
* função forward () é a substituição que define como encaminhar a propagação de entrada pelas camadas definidas do modelo.

* Pode ser definida a camada Linear para camadas totalmente conectadas e Conv2d para camadas convolucionais

* funções de ativação também podem ser definidas como camadas, como ReLU, Softmax e Sigmoid.


In [11]:
class MLP(Module):
    # Elementos do modelo
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # camada de entrada
        self.hidden1 = Linear(n_inputs, 10)
        # Inicialização da camada de entrada
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        # Ativação da camada de entrada
        self.act1 = ReLU()
        # segunda camada , entrada tem que ser do mesmo tamanho da saida da camada 1
        self.hidden2 = Linear(10, 8)
        # Inicialização da camada
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        # Ativação da camada
        self.act2 = ReLU()
        # camada de saída
        self.hidden3 = Linear(8, 1)
        # Inicialização da camada
        xavier_uniform_(self.hidden3.weight)
        # Ativação da camada
        self.act3 = Sigmoid()
 
    # propagação da entrada pelas camadas
    def forward(self, X):
        # entrada para primeira camada escondida
        X = self.hidden1(X)
        X = self.act1(X)
        # segunda camada escondida
        X = self.hidden2(X)
        X = self.act2(X)
        # terceira camada escondida
        X = self.hidden3(X)
        X = self.act3(X)
        return X

## O processo de treinamento 

* definir uma função de perda e um algoritmo de otimização.

* funções comuns de perda:
    * BCELoss: Perda de entropia cruzada binária para classificação binária.
    * CrossEntropyLoss: perda de entropia cruzada categórica para classificação de várias classes.
    * MSELoss: perda quadrática média para regressão.

* SGD é o algortimo de otimização padrão

## treinamento do modelo
* enumeração do DataLoader para o conjunto de dados de treinamento.

* loop para o número de épocas de treinamento. 
    * loop interno para os mini-lotes para a descida do gradiente estocástico.

## Cada atualização do modelo envolve o mesmo padrão geral:

* Limpando o último gradiente de erro.
* Uma passagem (forward) direta da entrada pelo modelo.
* Calculando a perda para a saída do modelo.
* Retropropagando o erro através do modelo.
* Atualize o modelo em um esforço para reduzir a perda. (otimização)

# rotina para treinar o modelo

In [12]:
def train_model(train_dl, model):
    # define loss
    criterion = BCELoss()
    # define otimizador
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # loop por épocas
    for epoch in range(100):
        # Loop em conjunto de mini-batches
        for i, (inputs, targets) in enumerate(train_dl):
            # zera os gradientes do batches
            optimizer.zero_grad()
            # predição do batch
            yhat = model(inputs)
            # calcula loss
            loss = criterion(yhat, targets)
            print("epoca " , epoch, " loss ", loss)
            # Retroprapagando erros 
            loss.backward()
            # atualiza pesos (otimização )
            optimizer.step()
            #print(loss)

## Avaliação do modelo

In [13]:
def evaluate_model(test_dl, model):
    # Cria lista de preditos e reais
    predictions, actuals = list(), list()

    # percorre lista do dataloader de test
    for i, (inputs, targets) in enumerate(test_dl):
        # realiza predição
        yhat = model(inputs)
        # cria numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round valores da classe
        yhat = yhat.round()
        # armazena
        predictions.append(yhat)
        actuals.append(actual)
        
    predictions, actuals = vstack(predictions), vstack(actuals)
    # usa sklearn para calcular acurácia
    acc = accuracy_score(actuals, predictions)
    return acc

## Recebe uma linha do dataset e calcula a predição

In [14]:
def predict(row, model):
    # converte linha do dataset para tensor
    row = Tensor([row])
    # predição
    yhat = model(row)
    # retorna valor predito
    yhat = yhat.detach().numpy()
    return yhat

In [15]:
# Prepara os dados
path = 'ionosphere.csv'
#path = '/home/silvio/git/exercicio-DL/veltec_clas.csv'

train_dl, test_dl = prepare_data(path)

print(len(train_dl.dataset), len(test_dl.dataset))

# criar modelo
model = MLP(34)
# treina o modelo
train_model(train_dl, model)
# avalia
acc = evaluate_model(test_dl, model)
print('Acuracia: %.3f' % acc)
# testa uma predição
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

235 116
epoca  0  loss  tensor(0.8329, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.7796, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.8055, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.7685, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.6595, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.7205, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.7020, grad_fn=<BinaryCrossEntropyBackward>)
epoca  0  loss  tensor(0.5438, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.5252, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.6717, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.7138, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.6972, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.5173, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.4965, grad_fn=<BinaryCrossEntropyBackward>)
epoca  1  loss  tensor(0.6

epoca  27  loss  tensor(0.1040, grad_fn=<BinaryCrossEntropyBackward>)
epoca  27  loss  tensor(0.1880, grad_fn=<BinaryCrossEntropyBackward>)
epoca  27  loss  tensor(0.1246, grad_fn=<BinaryCrossEntropyBackward>)
epoca  27  loss  tensor(0.1731, grad_fn=<BinaryCrossEntropyBackward>)
epoca  27  loss  tensor(0.2609, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1879, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1627, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1781, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1196, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.0470, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1255, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.1236, grad_fn=<BinaryCrossEntropyBackward>)
epoca  28  loss  tensor(0.2280, grad_fn=<BinaryCrossEntropyBackward>)
epoca  29  loss  tensor(0.0920, grad_fn=<BinaryCrossEntropyBackward>)
epoca  29  loss  ten

epoca  56  loss  tensor(0.0400, grad_fn=<BinaryCrossEntropyBackward>)
epoca  56  loss  tensor(0.1276, grad_fn=<BinaryCrossEntropyBackward>)
epoca  56  loss  tensor(0.0476, grad_fn=<BinaryCrossEntropyBackward>)
epoca  56  loss  tensor(0.0496, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0708, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.1696, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0366, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0441, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0285, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.1461, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0433, grad_fn=<BinaryCrossEntropyBackward>)
epoca  57  loss  tensor(0.0356, grad_fn=<BinaryCrossEntropyBackward>)
epoca  58  loss  tensor(0.0379, grad_fn=<BinaryCrossEntropyBackward>)
epoca  58  loss  tensor(0.0392, grad_fn=<BinaryCrossEntropyBackward>)
epoca  58  loss  ten

epoca  85  loss  tensor(0.0107, grad_fn=<BinaryCrossEntropyBackward>)
epoca  85  loss  tensor(0.1378, grad_fn=<BinaryCrossEntropyBackward>)
epoca  85  loss  tensor(0.0265, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0946, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0136, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0779, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.1132, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0238, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0230, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0361, grad_fn=<BinaryCrossEntropyBackward>)
epoca  86  loss  tensor(0.0327, grad_fn=<BinaryCrossEntropyBackward>)
epoca  87  loss  tensor(0.0166, grad_fn=<BinaryCrossEntropyBackward>)
epoca  87  loss  tensor(0.0550, grad_fn=<BinaryCrossEntropyBackward>)
epoca  87  loss  tensor(0.0175, grad_fn=<BinaryCrossEntropyBackward>)
epoca  87  loss  ten